In [5]:
import pyodbc
import pandas as pd
import numpy as np

In [ ]:
server = 'DESKTOP-R92IV14\BI2'
database = 'TSQL2012'
username = 'sa'
password = '9090'


conn_str = f'DRIVER={{ODBC Driver 17 for SQL Server}};SERVER={server};DATABASE={database};UID={username};PWD={password}'


conn = pyodbc.connect(conn_str)


In [7]:
query = "SELECT * FROM Sales.Orders" 
query2 = "SELECT * FROM Sales.OrderDetails"
orders = pd.read_sql(query, conn)
orderDetails = pd.read_sql(query2 , conn)


C:\Users\Aria\AppData\Local\Temp\ipykernel_36100\1667081855.py:3: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orders = pd.read_sql(query, conn)
C:\Users\Aria\AppData\Local\Temp\ipykernel_36100\1667081855.py:4: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  orderDetails = pd.read_sql(query2 , conn)


In [11]:
orders.head(4)

,orderid,custid,empid,orderdate,requireddate,shippeddate,shipperid,freight,shipname,shipaddress,shipcity,shipregion,shippostalcode,shipcountry
0,10248,85,5,2006-07-04,2006-08-01,2006-07-16,3,32.38,Ship to 85-B,6789 rue de l'Abbaye,Reims,None,10345,France
1,10249,79,6,2006-07-05,2006-08-16,2006-07-10,1,11.61,Ship to 79-C,Luisenstr. 9012,Münster,None,10328,Germany
2,10250,34,4,2006-07-08,2006-08-05,2006-07-12,2,65.83,Destination SCQXA,"Rua do Paço, 7890",Rio de Janeiro,RJ,10195,Brazil
3,10251,84,3,2006-07-08,2006-08-05,2006-07-15,1,41.34,Ship to 84-A,"3456, rue du Commerce",Lyon,None,10342,France


In [12]:
orderDetails.head(4)

,orderid,productid,unitprice,qty,discount
0,10248,11,14.0,12,0.0
1,10248,42,9.8,10,0.0
2,10248,72,34.8,5,0.0
3,10249,14,18.6,9,0.0


In [22]:
temp = orders.merge(orderDetails ,how = "inner" , on="orderid")

In [23]:
temp.head(2)

,orderid,custid,empid,orderdate,requireddate,shippeddate,shipperid,freight,shipname,shipaddress,shipcity,shipregion,shippostalcode,shipcountry,productid,unitprice,qty,discount
0,10248,85,5,2006-07-04,2006-08-01,2006-07-16,3,32.38,Ship to 85-B,6789 rue de l'Abbaye,Reims,None,10345,France,11,14.0,12,0.0
1,10248,85,5,2006-07-04,2006-08-01,2006-07-16,3,32.38,Ship to 85-B,6789 rue de l'Abbaye,Reims,None,10345,France,42,9.8,10,0.0


In [52]:
temp['Monetary'] = temp['qty'] * temp['unitprice']

result = temp.groupby('custid').agg(
    Monetary=('Monetary', 'sum'),       
    Frequency=('orderdate', 'max') ,  
    Recancy = ('orderid' , 'nunique')
).reset_index()

result['r'] = pd.qcut(result['Recancy'] , q = 4 , labels=False) + 1
result['f'] = pd.qcut(result['Frequency'] , q = 4 , labels=False) + 1
result['m'] = pd.qcut(result['Monetary'] , q = 4 , labels=False) + 1

result.sort_values(by = ['r' , 'f' , 'm'] , inplace=False)



,custid,Monetary,Frequency,Recancy,r,f,m
1,2,1402.95,2008-03-04,4,1,1,1
12,13,100.80,2006-07-18,1,1,1,1
15,16,1719.10,2008-01-23,3,1,1,1
17,18,1615.90,2008-02-16,4,1,1,1
27,29,836.70,2008-03-05,5,1,1,1
...,...,...,...,...,...,...,...
42,44,21282.02,2008-05-05,15,4,4,4
59,62,30226.10,2008-05-04,13,4,4,4
62,65,52245.90,2008-05-06,18,4,4,4
68,71,115673.39,2008-05-01,31,4,4,4


In [4]:
conn.close() # بستن اتصال